In [1]:
import numpy as np
import os
import pandas as pd

# Import the required ArcGIS API for Python modules
import arcgis
from arcgis.gis import GIS
gis = arcgis.GIS()
from arcgis.geoanalytics import manage_data

from IPython.display import display, Markdown

In [2]:
dirWork         = os.getcwd()
dirInput        = os.path.join(dirWork, r'input'         )
dirIntermediate = os.path.join(dirWork, r'intermediate'  )
dirResults      = os.path.join(dirWork, r'results'       )
dirParams       = os.path.join(dirWork, r'params'        )
dirWebApp       = os.path.join(dirResults, r'webapp_data')

In [3]:
#intermediate output
dfFactors = pd.read_csv(os.path.join(dirIntermediate,'CCS_Factors_AllGroupings.csv'))
dfSegFac  = pd.read_csv(os.path.join(dirIntermediate, 'SegmentsMatchedToFactorGroups.csv'))
#dfSegFacWithTrk = pd.read_csv(os.path.join(dirIntermediate, r'SegmentTruckFactors.csv'))

#Start with UDOT Base
#sdfSegments = pd.DataFrame.spatial.from_featureclass(os.path.join(dirWork,  r"arcgis\UDOT-Segments.gdb\Master_Segs_20210811"))

#Start with Modified Base - and strip down of factor fields
sdfSegments = pd.DataFrame.spatial.from_featureclass(os.path.join(dirInput, r"master\Final_Segments\Final_Segments.shp"))
#sdfSegments = sdfSegments.drop(columns=['FAC_MON'  ,'FAC_TUE'  ,'FAC_WED'  ,'FAC_THU'  ,'FAC_FRI'  ,'FAC_SAT'  ,'FAC_SUN'  ,'FAC_WDAVG','FAC_WEAVG','FAC_WEMAX','FAC_JAN'  ,'FAC_FEB'  ,'FAC_MAR'  ,'FAC_APR'  ,'FAC_MAY'  ,'FAC_JUN'  ,'FAC_JUL'  ,'FAC_AUG'  ,'FAC_SEP'  ,'FAC_OCT'  ,'FAC_NOV'  ,'FAC_DEC'  ,'FAC_WIN'   ,'FAC_SPR'   ,'FAC_SUM'   ,'FAC_FAL'  ,'FAC_MAXMO','FAC_MAX'  ,'FACMANADJ','CUTRUCKS','SUTRUCKS'])


In [4]:
dfSegFac

,SEGID,AADT2019,AREATYPE,FTGROUP,FACGEO,STATIONGROUP
0,0006_000.0,415.0,1,Arterial,Statewide,COP
1,0006_000.7,415.0,1,Arterial,Statewide,COP
2,0006_016.0,415.0,1,Arterial,Statewide,COP
3,0006_046.0,372.0,1,Arterial,Statewide,COP
4,0006_060.2,372.0,1,Arterial,Statewide,COP
...,...,...,...,...,...,...
8717,0056_060.6,16915.0,3,Arterial,Statewide,COT
8718,0056_060.5,16915.0,3,Arterial,Statewide,COT
8719,0056_060.4,13242.0,3,Arterial,Statewide,COT
8720,1760_000.0,713.0,3,Arterial,Statewide,COT


In [5]:
#parameters
prmInterpolate = pd.read_csv(os.path.join(dirParams,'segment_factors_interpolate.csv'))

In [6]:
dFactorGroupsToPublish = {
                           'SEASONGROUP': ['Year'     ,'Year'     ,'Year'     ,'Year'     ,'Year'     ,'Year'     ,'Year'     ,'Year'     ,'Year'     ,'Year'     ,'M01-Jan'  ,'M02-Feb'  ,'M03-Mar'  ,'M04-Apr'  ,'M05-May'  ,'M06-Jun'  ,'M07-Jul'  ,'M08-Aug'  ,'M09-Sep'  ,'M10-Oct'  ,'M11-Nov'  ,'M12-Dec'  ,'S01-Winter','S02-Spring','S03-Summer','S04-Fall' ,'Year'     ,'Year'     ],
                           'YEARGROUP'  : ['2015-2019','2015-2019','2015-2019','2015-2019','2015-2019','2015-2019','2015-2019','2015-2019','2015-2019','2015-2019','2015-2019','2015-2019','2015-2019','2015-2019','2015-2019','2015-2019','2015-2019','2015-2019','2015-2019','2015-2019','2015-2019','2015-2019','2015-2019' ,'2015-2019' ,'2015-2019' ,'2015-2019','2015-2019','2015-2019'],
                           'FACTORNAME' : ['FAC_MON'  ,'FAC_TUE'  ,'FAC_WED'  ,'FAC_THU'  ,'FAC_FRI'  ,'FAC_SAT'  ,'FAC_SUN'  ,'FAC_WDAVG','FAC_WEAVG','FAC_WEMAX','FAC_JAN'  ,'FAC_FEB'  ,'FAC_MAR'  ,'FAC_APR'  ,'FAC_MAY'  ,'FAC_JUN'  ,'FAC_JUL'  ,'FAC_AUG'  ,'FAC_SEP'  ,'FAC_OCT'  ,'FAC_NOV'  ,'FAC_DEC'  ,'FAC_WIN'   ,'FAC_SPR'   ,'FAC_SUM'   ,'FAC_FAL'  ,'FAC_MAXMO','FAC_MAX'  ]
                         }

dStationGroupsToPublish = {'STATIONGROUP': ['CO0',
                                            'CO1',
                                            'CO2',
                                            'CO3',
                                            'CO7',
                                            'CO4',
                                            'CO5',
                                            'CO6',
                                            'CO8',
                                            'CO9',
                                            'COA',
                                            'COB',
                                            'COC',
                                            'COD',
                                            'COY',
                                            'COE',
                                            'COZ',
                                            'COF',
                                            'COG',
                                            'COH',
                                            'COI',
                                            'COJ',
                                            'COK',
                                            'COL',
                                            'COM',
                                            'CON',
                                            'COO',
                                            'COP',
                                            'COQ',
                                            'COR',
                                            'COS',
                                            'COT',
                                            'COU',
                                            'XX3',
                                            'COV',
                                            'COW',
                                            'COX']}

dfFactorGroupsToPublish = pd.DataFrame(dFactorGroupsToPublish)
display(dfFactorGroupsToPublish)

dfStationGroupsToPublish = pd.DataFrame(dStationGroupsToPublish)
display(dfStationGroupsToPublish)

dfToPublish = pd.DataFrame.merge(dfStationGroupsToPublish,dfFactorGroupsToPublish,how='cross')

dfFactorsToPublish = pd.DataFrame.merge(dfFactors,dfToPublish,on=('STATIONGROUP','SEASONGROUP','YEARGROUP','FACTORNAME'))
dfFactorsToPublish = dfFactorsToPublish[['STATIONGROUP','FACTORNAME','FACTOR']]
display(dfFactorsToPublish)



,SEASONGROUP,YEARGROUP,FACTORNAME
0,Year,2015-2019,FAC_MON
1,Year,2015-2019,FAC_TUE
2,Year,2015-2019,FAC_WED
3,Year,2015-2019,FAC_THU
4,Year,2015-2019,FAC_FRI
5,Year,2015-2019,FAC_SAT
6,Year,2015-2019,FAC_SUN
7,Year,2015-2019,FAC_WDAVG
8,Year,2015-2019,FAC_WEAVG
9,Year,2015-2019,FAC_WEMAX


,STATIONGROUP
0,CO0
1,CO1
2,CO2
3,CO3
4,CO7
5,CO4
6,CO5
7,CO6
8,CO8
9,CO9


,STATIONGROUP,FACTORNAME,FACTOR
0,CO0,FAC_JAN,0.320612
1,CO1,FAC_JAN,0.432402
2,CO2,FAC_JAN,1.140254
3,CO3,FAC_JAN,0.385754
4,CO4,FAC_JAN,0.751806
...,...,...,...
1031,XX3,FAC_TUE,1.146486
1032,XX3,FAC_WDAVG,1.156760
1033,XX3,FAC_WEAVG,0.631137
1034,XX3,FAC_WED,1.158740


In [7]:
#pivot for final table
dfFactorsToPublishStationGroup_pivot = dfFactorsToPublish.pivot(index=('STATIONGROUP'),columns='FACTORNAME',values='FACTOR')

dfFactorsToPublishStationGroup_pivot = dfFactorsToPublishStationGroup_pivot.reset_index()

# Add a row with zeros

# Create a new row
new_row = {'STATIONGROUP': 'TRN'}
for col in dfFactorsToPublishStationGroup_pivot.columns:
    if col != 'STATIONGROUP':
        new_row[col] = 0

# Append the new row to the dataframe
dfFactorsToPublishStationGroup_pivot = dfFactorsToPublishStationGroup_pivot.append(new_row, ignore_index=True)

# reverse pivot for long table:
dfFactorsToPublish = dfFactorsToPublishStationGroup_pivot.melt(id_vars=('STATIONGROUP'), var_name="FACTORNAME", value_name="FACTOR")

dfFactorsToPublishStationGroup_pivot.to_csv(os.path.join(dirResults, r'CCSFactors.csv'),index=False)
#display(dfFactorsToPublishStationGroup_pivot)


In [8]:
dfSegFacToPublish = pd.DataFrame.merge(dfSegFac,dfFactorsToPublish,on='STATIONGROUP',how="left")
display(dfSegFacToPublish[dfSegFacToPublish['SEGID'].str.contains('UTA')])
display(dfSegFacToPublish)

,SEGID,AADT2019,AREATYPE,FTGROUP,FACGEO,STATIONGROUP,FACTORNAME,FACTOR
193340,UTA_9544,0.0,4,Transit,Statewide,TRN,FAC_APR,0.0
193341,UTA_9544,0.0,4,Transit,Statewide,TRN,FAC_AUG,0.0
193342,UTA_9544,0.0,4,Transit,Statewide,TRN,FAC_DEC,0.0
193343,UTA_9544,0.0,4,Transit,Statewide,TRN,FAC_FAL,0.0
193344,UTA_9544,0.0,4,Transit,Statewide,TRN,FAC_FEB,0.0
...,...,...,...,...,...,...,...,...
195687,UTA_7576,0.0,4,Transit,Statewide,TRN,FAC_WDAVG,0.0
195688,UTA_7576,0.0,4,Transit,Statewide,TRN,FAC_WEAVG,0.0
195689,UTA_7576,0.0,4,Transit,Statewide,TRN,FAC_WED,0.0
195690,UTA_7576,0.0,4,Transit,Statewide,TRN,FAC_WEMAX,0.0


,SEGID,AADT2019,AREATYPE,FTGROUP,FACGEO,STATIONGROUP,FACTORNAME,FACTOR
0,0006_000.0,415.0,1,Arterial,Statewide,COP,FAC_APR,1.036642
1,0006_000.0,415.0,1,Arterial,Statewide,COP,FAC_AUG,1.089736
2,0006_000.0,415.0,1,Arterial,Statewide,COP,FAC_DEC,0.882206
3,0006_000.0,415.0,1,Arterial,Statewide,COP,FAC_FAL,1.031575
4,0006_000.0,415.0,1,Arterial,Statewide,COP,FAC_FEB,0.850375
...,...,...,...,...,...,...,...,...
244211,Dixie_5134,0.0,1,Arterial,Washington,COV,FAC_WDAVG,1.071159
244212,Dixie_5134,0.0,1,Arterial,Washington,COV,FAC_WEAVG,0.789826
244213,Dixie_5134,0.0,1,Arterial,Washington,COV,FAC_WED,1.062677
244214,Dixie_5134,0.0,1,Arterial,Washington,COV,FAC_WEMAX,1.011186


In [9]:
#manual interpolatation

display(prmInterpolate)

dfFactorsToPublish_InterpolateFrom = pd.DataFrame.merge(prmInterpolate,dfSegFacToPublish,left_on='SEGIDFROM',right_on='SEGID')
dfFactorsToPublish_InterpolateFrom = dfFactorsToPublish_InterpolateFrom[['SEGIDTOREV','FACTORNAME','FACTOR','PORTIONFROM']]
dfFactorsToPublish_InterpolateFrom = dfFactorsToPublish_InterpolateFrom.rename(columns={'FACTOR':'FACTORFROM'})
display(dfFactorsToPublish_InterpolateFrom)

dfFactorsToPublish_InterpolateTo   = pd.DataFrame.merge(prmInterpolate,dfSegFacToPublish,left_on='SEGIDTO'  ,right_on='SEGID')
dfFactorsToPublish_InterpolateTo   = dfFactorsToPublish_InterpolateTo  [['SEGIDTOREV','FACTORNAME','FACTOR','PORTIONTO']]
dfFactorsToPublish_InterpolateTo   = dfFactorsToPublish_InterpolateTo.rename(columns={'FACTOR':'FACTORTO'})
display(dfFactorsToPublish_InterpolateTo)

dfFactorsToPublish_Interpolate = pd.DataFrame.merge(dfFactorsToPublish_InterpolateFrom,dfFactorsToPublish_InterpolateTo,on=('SEGIDTOREV','FACTORNAME'))

dfFactorsToPublish_Interpolate['PF'] = dfFactorsToPublish_Interpolate['PORTIONFROM']/(dfFactorsToPublish_Interpolate['PORTIONTO']+dfFactorsToPublish_Interpolate['PORTIONFROM'])
dfFactorsToPublish_Interpolate['PT'] = dfFactorsToPublish_Interpolate['PORTIONTO'  ]/(dfFactorsToPublish_Interpolate['PORTIONTO']+dfFactorsToPublish_Interpolate['PORTIONFROM'])
dfFactorsToPublish_Interpolate['FACTORINTERPOLATED'] = dfFactorsToPublish_Interpolate['FACTORFROM'] * dfFactorsToPublish_Interpolate['PF'] + dfFactorsToPublish_Interpolate['FACTORTO'] * dfFactorsToPublish_Interpolate['PT']
display(dfFactorsToPublish_Interpolate)

dfFactorsToPublish_Interpolate = dfFactorsToPublish_Interpolate[['SEGIDTOREV','FACTORNAME','FACTORINTERPOLATED']]
dfFactorsToPublish_Interpolate = dfFactorsToPublish_Interpolate.rename(columns={'SEGIDTOREV':'SEGID'})
dfFactorsToPublish_Interpolate

,ID,SEGIDTOREV,SEGIDFROM,PORTIONFROM,SEGIDTO,PORTIONTO
0,1,0015_225.3,0015_222.8,11,0015_261.8,1
1,2,0015_225.8,0015_222.8,11,0015_261.8,1
2,3,0015_228.1,0015_222.8,10,0015_261.8,2
3,4,0015_230.0,0015_222.8,10,0015_261.8,2
4,5,0015_233.0,0015_222.8,10,0015_261.8,2
...,...,...,...,...,...,...
64,65,0006_175.6,0006_178.8,5,0006_173.4,6
65,66,0006_174.9,0006_178.8,4,0006_173.4,7
66,67,0006_174.4,0006_178.8,3,0006_173.4,8
67,68,0006_174.0,0006_178.8,2,0006_173.4,9


,SEGIDTOREV,FACTORNAME,FACTORFROM,PORTIONFROM
0,0015_225.3,FAC_APR,1.067708,11
1,0015_225.3,FAC_AUG,1.075263,11
2,0015_225.3,FAC_DEC,0.890783,11
3,0015_225.3,FAC_FAL,1.006220,11
4,0015_225.3,FAC_FEB,0.835322,11
...,...,...,...,...
1927,0006_173.7,FAC_WDAVG,0.959262,1
1928,0006_173.7,FAC_WEAVG,1.014440,1
1929,0006_173.7,FAC_WED,0.952876,1
1930,0006_173.7,FAC_WEMAX,1.107057,1


,SEGIDTOREV,FACTORNAME,FACTORTO,PORTIONTO
0,0015_225.3,FAC_APR,1.018984,1
1,0015_225.3,FAC_AUG,1.039075,1
2,0015_225.3,FAC_DEC,0.989176,1
3,0015_225.3,FAC_FAL,1.013135,1
4,0015_225.3,FAC_FEB,0.938123,1
...,...,...,...,...
1927,0006_173.7,FAC_WDAVG,1.094640,10
1928,0006_173.7,FAC_WEAVG,0.758306,10
1929,0006_173.7,FAC_WED,1.092516,10
1930,0006_173.7,FAC_WEMAX,0.887302,10


,SEGIDTOREV,FACTORNAME,FACTORFROM,PORTIONFROM,FACTORTO,PORTIONTO,PF,PT,FACTORINTERPOLATED
0,0015_225.3,FAC_APR,1.067708,11,1.018984,1,0.916667,0.083333,1.063648
1,0015_225.3,FAC_AUG,1.075263,11,1.039075,1,0.916667,0.083333,1.072248
2,0015_225.3,FAC_DEC,0.890783,11,0.989176,1,0.916667,0.083333,0.898982
3,0015_225.3,FAC_FAL,1.006220,11,1.013135,1,0.916667,0.083333,1.006796
4,0015_225.3,FAC_FEB,0.835322,11,0.938123,1,0.916667,0.083333,0.843888
...,...,...,...,...,...,...,...,...,...
1927,0006_173.7,FAC_WDAVG,0.959262,1,1.094640,10,0.090909,0.909091,1.082333
1928,0006_173.7,FAC_WEAVG,1.014440,1,0.758306,10,0.090909,0.909091,0.781591
1929,0006_173.7,FAC_WED,0.952876,1,1.092516,10,0.090909,0.909091,1.079821
1930,0006_173.7,FAC_WEMAX,1.107057,1,0.887302,10,0.090909,0.909091,0.907280


,SEGID,FACTORNAME,FACTORINTERPOLATED
0,0015_225.3,FAC_APR,1.063648
1,0015_225.3,FAC_AUG,1.072248
2,0015_225.3,FAC_DEC,0.898982
3,0015_225.3,FAC_FAL,1.006796
4,0015_225.3,FAC_FEB,0.843888
...,...,...,...
1927,0006_173.7,FAC_WDAVG,1.082333
1928,0006_173.7,FAC_WEAVG,0.781591
1929,0006_173.7,FAC_WED,1.079821
1930,0006_173.7,FAC_WEMAX,0.907280


In [10]:
#update with interpolated values

dfSegFacToPublishWithInterpolation = pd.DataFrame.merge(dfSegFacToPublish,dfFactorsToPublish_Interpolate,on=('SEGID','FACTORNAME'),how='left')
dfSegFacToPublishWithInterpolation = dfSegFacToPublishWithInterpolation.fillna(0)
dfSegFacToPublishWithInterpolation.loc[(dfSegFacToPublishWithInterpolation['FACTORINTERPOLATED']>0),'FACTOR'] = dfSegFacToPublishWithInterpolation['FACTORINTERPOLATED']

dfSegFacToPublishWithInterpolation['FACMANADJ'] = 0
dfSegFacToPublishWithInterpolation.loc[(dfSegFacToPublishWithInterpolation['FACTORINTERPOLATED']>0),'FACMANADJ'] = 1


dfSegFacToPublishWithInterpolation['FACTOR'] = round(dfSegFacToPublishWithInterpolation['FACTOR'],4)

display(dfSegFacToPublishWithInterpolation[dfSegFacToPublishWithInterpolation['FACTORINTERPOLATED']>0])

display(dfSegFacToPublishWithInterpolation)

,SEGID,AADT2019,AREATYPE,FTGROUP,FACGEO,STATIONGROUP,FACTORNAME,FACTOR,FACTORINTERPOLATED,FACMANADJ
4256,0015_225.3,25196.0,1,Freeway,I15,CO4,FAC_APR,1.0636,1.063648,1
4257,0015_225.3,25196.0,1,Freeway,I15,CO4,FAC_AUG,1.0722,1.072248,1
4258,0015_225.3,25196.0,1,Freeway,I15,CO4,FAC_DEC,0.8990,0.898982,1
4259,0015_225.3,25196.0,1,Freeway,I15,CO4,FAC_FAL,1.0068,1.006796,1
4260,0015_225.3,25196.0,1,Freeway,I15,CO4,FAC_FEB,0.8439,0.843888,1
...,...,...,...,...,...,...,...,...,...,...
211759,0018_040.3,2129.0,1,Arterial,Washington,COV,FAC_WDAVG,1.0015,1.001454,1
211760,0018_040.3,2129.0,1,Arterial,Washington,COV,FAC_WEAVG,0.9367,0.936715,1
211761,0018_040.3,2129.0,1,Arterial,Washington,COV,FAC_WED,0.9943,0.994350,1
211762,0018_040.3,2129.0,1,Arterial,Washington,COV,FAC_WEMAX,1.0542,1.054190,1


,SEGID,AADT2019,AREATYPE,FTGROUP,FACGEO,STATIONGROUP,FACTORNAME,FACTOR,FACTORINTERPOLATED,FACMANADJ
0,0006_000.0,415.0,1,Arterial,Statewide,COP,FAC_APR,1.0366,0.0,0
1,0006_000.0,415.0,1,Arterial,Statewide,COP,FAC_AUG,1.0897,0.0,0
2,0006_000.0,415.0,1,Arterial,Statewide,COP,FAC_DEC,0.8822,0.0,0
3,0006_000.0,415.0,1,Arterial,Statewide,COP,FAC_FAL,1.0316,0.0,0
4,0006_000.0,415.0,1,Arterial,Statewide,COP,FAC_FEB,0.8504,0.0,0
...,...,...,...,...,...,...,...,...,...,...
244211,Dixie_5134,0.0,1,Arterial,Washington,COV,FAC_WDAVG,1.0712,0.0,0
244212,Dixie_5134,0.0,1,Arterial,Washington,COV,FAC_WEAVG,0.7898,0.0,0
244213,Dixie_5134,0.0,1,Arterial,Washington,COV,FAC_WED,1.0627,0.0,0
244214,Dixie_5134,0.0,1,Arterial,Washington,COV,FAC_WEMAX,1.0112,0.0,0


In [11]:
#pivot for final table

dfFactorsToPublish_pivot = dfSegFacToPublishWithInterpolation.pivot(index=('SEGID','STATIONGROUP','FACMANADJ'),columns='FACTORNAME',values='FACTOR')

#change maximum month to integer
dfFactorsToPublish_pivot['FAC_MAXMO'] = dfFactorsToPublish_pivot['FAC_MAXMO'].astype(int)

dfFactorsToPublish_pivot = dfFactorsToPublish_pivot.reset_index()
display(dfFactorsToPublish_pivot)

FACTORNAME,SEGID,STATIONGROUP,FACMANADJ,FAC_APR,FAC_AUG,FAC_DEC,FAC_FAL,FAC_FEB,FAC_FRI,FAC_JAN,...,FAC_SPR,FAC_SUM,FAC_SUN,FAC_THU,FAC_TUE,FAC_WDAVG,FAC_WEAVG,FAC_WED,FAC_WEMAX,FAC_WIN
0,0006_000.0,COP,0,1.0366,1.0897,0.8822,1.0316,0.8504,1.1505,0.8159,...,1.0276,1.0914,0.8819,1.0120,0.9628,0.9840,0.9734,0.9773,1.0649,0.8495
1,0006_000.7,COP,0,1.0366,1.0897,0.8822,1.0316,0.8504,1.1505,0.8159,...,1.0276,1.0914,0.8819,1.0120,0.9628,0.9840,0.9734,0.9773,1.0649,0.8495
2,0006_016.0,COP,0,1.0366,1.0897,0.8822,1.0316,0.8504,1.1505,0.8159,...,1.0276,1.0914,0.8819,1.0120,0.9628,0.9840,0.9734,0.9773,1.0649,0.8495
3,0006_046.0,COP,0,1.0366,1.0897,0.8822,1.0316,0.8504,1.1505,0.8159,...,1.0276,1.0914,0.8819,1.0120,0.9628,0.9840,0.9734,0.9773,1.0649,0.8495
4,0006_060.2,COP,0,1.0366,1.0897,0.8822,1.0316,0.8504,1.1505,0.8159,...,1.0276,1.0914,0.8819,1.0120,0.9628,0.9840,0.9734,0.9773,1.0649,0.8495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8717,WFRC_8262,COT,0,1.0146,1.0396,0.9616,1.0243,0.9471,1.1418,0.9024,...,1.0104,1.0283,0.6061,1.1071,1.0813,1.0924,0.7653,1.0887,0.9245,0.9370
8718,WFRC_8263,COT,0,1.0146,1.0396,0.9616,1.0243,0.9471,1.1418,0.9024,...,1.0104,1.0283,0.6061,1.1071,1.0813,1.0924,0.7653,1.0887,0.9245,0.9370
8719,WFRC_8264,COT,0,1.0146,1.0396,0.9616,1.0243,0.9471,1.1418,0.9024,...,1.0104,1.0283,0.6061,1.1071,1.0813,1.0924,0.7653,1.0887,0.9245,0.9370
8720,WFRC_8265,COU,0,1.0142,1.0223,0.9907,1.0095,0.9705,1.1450,0.9346,...,1.0117,1.0135,0.6293,1.1063,1.0851,1.0946,0.7583,1.0925,0.8873,0.9653


In [12]:
#display(dfSegFacWithTrk.columns)
#dfFinalFactors = pd.DataFrame.merge(dfFactorsToPublish_pivot,dfSegFacWithTrk,on=('SEGID','STATIONGROUP'))
#dfFinalFactors = dfFinalFactors.drop(columns='AADT2019')
#dfFinalFactors
dfFinalFactors = dfFactorsToPublish_pivot

In [13]:
from datetime import datetime
#get today's date in yyyymmdd format
today = datetime.today().strftime('%Y%m%d')

sdfSegmentsWithFinalFactors = pd.DataFrame.merge(sdfSegments,dfFinalFactors,on='SEGID')
sdfSegmentsWithFinalFactors = sdfSegmentsWithFinalFactors.sort_values(by=['SEGID'])
sdfSegmentsWithFinalFactors = sdfSegmentsWithFinalFactors.drop(columns=['FID'])
sdfSegmentsWithFinalFactors = sdfSegmentsWithFinalFactors.fillna(0)

#Rename STATIONGROUP to CCSGROUP
sdfSegmentsWithFinalFactors = sdfSegmentsWithFinalFactors.rename(columns={"STATIONGROUP": "CCSGROUP19"})

sdfSegmentsWithFinalFactors.spatial.to_featureclass(location=os.path.join(dirResults, r'layers\Master_Segs_withFactors_' + today + '.shp'),sanitize_columns=False)
display(sdfSegmentsWithFinalFactors)

,SEGID,BMP,EMP,PLANAREA,AADT2019,SUBAREAID,DISTANCE,CO_FIPS,F_AREA,SHAPE,...,FAC_SPR,FAC_SUM,FAC_SUN,FAC_THU,FAC_TUE,FAC_WDAVG,FAC_WEAVG,FAC_WED,FAC_WEMAX,FAC_WIN
0,0006_000.0,0.000,0.665,UDOT,415.0,0,0.666642,27,UDOT,"{""paths"": [[[236177.7000000002, 4327541.25], [...",...,1.0276,1.0914,0.8819,1.0120,0.9628,0.9840,0.9734,0.9773,1.0649,0.8495
1,0006_000.7,0.665,16.022,UDOT,415.0,0,15.369870,27,UDOT,"{""paths"": [[[237241.1799999997, 4327399.720000...",...,1.0276,1.0914,0.8819,1.0120,0.9628,0.9840,0.9734,0.9773,1.0649,0.8495
2,0006_016.0,16.022,46.017,UDOT,415.0,0,30.002021,27,UDOT,"{""paths"": [[[261403.90000000037, 4327045.4], [...",...,1.0276,1.0914,0.8819,1.0120,0.9628,0.9840,0.9734,0.9773,1.0649,0.8495
3,0006_046.0,46.017,60.218,UDOT,372.0,0,14.194335,27,UDOT,"{""paths"": [[[305380.16000000015, 4325741.85], ...",...,1.0276,1.0914,0.8819,1.0120,0.9628,0.9840,0.9734,0.9773,1.0649,0.8495
4,0006_060.2,60.218,77.545,UDOT,372.0,0,17.323272,27,UDOT,"{""paths"": [[[324618, 4337936.9], [324647.40000...",...,1.0276,1.0914,0.8819,1.0120,0.9628,0.9840,0.9734,0.9773,1.0649,0.8495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6833,WFRC_8262,0.000,0.380,WFRC,6672.0,1,0.420119,35,WFRC,"{""paths"": [[[416468.1179999998, 4486835.126], ...",...,1.0104,1.0283,0.6061,1.1071,1.0813,1.0924,0.7653,1.0887,0.9245,0.9370
6835,WFRC_8263,0.000,0.380,WFRC,6672.0,1,0.470366,35,WFRC,"{""paths"": [[[416468.1174999997, 4486835.126], ...",...,1.0104,1.0283,0.6061,1.1071,1.0813,1.0924,0.7653,1.0887,0.9245,0.9370
6892,WFRC_8264,0.000,0.000,WFRC,0.0,1,0.448310,11,WFRC,"{""paths"": [[[421337.5999999996, 4538738.6], [4...",...,1.0104,1.0283,0.6061,1.1071,1.0813,1.0924,0.7653,1.0887,0.9245,0.9370
6896,WFRC_8265,0.000,0.000,WFRC,0.0,1,0.298847,35,WFRC,"{""paths"": [[[412956.6738, 4507985.040999999], ...",...,1.0117,1.0135,0.6293,1.1063,1.0851,1.0946,0.7583,1.0925,0.8873,0.9653


In [14]:
sdfSegmentsWithFinalFactors.columns

Index(['SEGID', 'BMP', 'EMP', 'PLANAREA', 'AADT2019', 'SUBAREAID', 'DISTANCE',
       'CO_FIPS', 'F_AREA', 'SHAPE', 'CCSGROUP19', 'FACMANADJ', 'FAC_APR',
       'FAC_AUG', 'FAC_DEC', 'FAC_FAL', 'FAC_FEB', 'FAC_FRI', 'FAC_JAN',
       'FAC_JUL', 'FAC_JUN', 'FAC_MAR', 'FAC_MAX', 'FAC_MAXMO', 'FAC_MAY',
       'FAC_MON', 'FAC_NOV', 'FAC_OCT', 'FAC_SAT', 'FAC_SEP', 'FAC_SPR',
       'FAC_SUM', 'FAC_SUN', 'FAC_THU', 'FAC_TUE', 'FAC_WDAVG', 'FAC_WEAVG',
       'FAC_WED', 'FAC_WEMAX', 'FAC_WIN'],
      dtype='object')

In [15]:
#export table to csv

dfSegmentsWithFinalFactors = sdfSegmentsWithFinalFactors.drop(columns=['SHAPE'])
dfSegmentsWithFinalFactors.to_csv(os.path.join(dirResults, r'Master_Segs_withFactors_' + today + '.csv'),index=False)

In [16]:
os.path.join(dirResults, r'Master_Segs_withFactors_' + today + '.csv')

'e:\\GitHub\\UDOT-Segment-Factors\\results\\Master_Segs_withFactors_20231027.csv'

In [17]:
sdfSegmentsWithFinalFactors.columns

Index(['SEGID', 'BMP', 'EMP', 'PLANAREA', 'AADT2019', 'SUBAREAID', 'DISTANCE',
       'CO_FIPS', 'F_AREA', 'SHAPE', 'CCSGROUP19', 'FACMANADJ', 'FAC_APR',
       'FAC_AUG', 'FAC_DEC', 'FAC_FAL', 'FAC_FEB', 'FAC_FRI', 'FAC_JAN',
       'FAC_JUL', 'FAC_JUN', 'FAC_MAR', 'FAC_MAX', 'FAC_MAXMO', 'FAC_MAY',
       'FAC_MON', 'FAC_NOV', 'FAC_OCT', 'FAC_SAT', 'FAC_SEP', 'FAC_SPR',
       'FAC_SUM', 'FAC_SUN', 'FAC_THU', 'FAC_TUE', 'FAC_WDAVG', 'FAC_WEAVG',
       'FAC_WED', 'FAC_WEMAX', 'FAC_WIN'],
      dtype='object')

In [18]:
sdfSegmentsWithFinalFactors['CCSGROUP19']

0       COP
1       COP
2       COP
3       COP
4       COP
       ... 
6833    COT
6835    COT
6892    COT
6896    COU
6904    COU
Name: CCSGROUP19, Length: 8722, dtype: object